In [126]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

import re
import os
import pickle
from enum import Enum

In [ ]:
"""
Oauth consent https://console.cloud.google.com/apis/credentials/consent?project=potent-ripple-377913
youtube = YouTube(credentials_file='', token_path='')
youtube.get_youtube() -> youtube_client
youtube.search_video(): query -> List[YouTubeVideo]
youtube.search_channel(): query -> List[YouTubeChannel]
youtube.search_playlist(): query -> List[PlayList]
youtube.find_video_by_id(): video_id -> YouTubeVideo
youtube.find_video_by_url(): video_url -> YouTubeVideo
youtube.utils
"""

In [127]:
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [128]:
class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    __TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIConstants.API_VERSION
    __SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.__credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.__api_token_path = self.__get_default_api_token_path()
        else:
            self.__api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.__api_token_path):
            with open(self.__api_token_path, "rb") as token:
                self.__credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            # save the credentials for the next run
            with open(self.__api_token_path, "wb") as token:
                pickle.dump(self.__credentials, token)

        return build(self.__API_SERVICE_NAME, self.__API_VERSION, credentials=self.__credentials)
    
    def authenticate(self):
        try:
            youtube_client =  self.__authenticate_youtube()
        except RefreshError as e:
            #delete token
            raise Exception('The token is expired. Kindly generate a new one.')
        else:
            return youtube_client

In [129]:
class YouTubeSearchQuery:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValueError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

        
class YouTubeVideoSearchQuery(YouTubeSearchQuery):
    pass

In [130]:
class YouTubeSearchType:
    VIDEO = 'video'
    CHANNEL = 'channel'
    PLAYLIST = 'playlist'

In [131]:
class YouTubeSearch:
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, *args, **kwargs):
        pass

In [132]:
class YouTubeVideoStats:
    def __init__(self, viewCount: int, likeCount: int, commentCount: int):
        self.__view_count = int(viewCount)
        self.__like_count = int(likeCount)
        self.__comment_count = int(commentCount)
        
    def get_video_stats(self):
        video_stats = {
            'view_count': self.__view_count,
            'like_count': self.__like_count,
            'comment_count': self.__comment_count
        }
        return video_stats

In [133]:
class YouTubeVideoDetails:
    def __init__(self, id: str, channelId: str, title: str, channelTitle: str, 
                 description: str, tags: list[str], duration: str, licensedContent: bool):
        self.__id = id
        self.__channel_id = channelId
        self.__title = title
        self.__channel_title = channelTitle
        self.__description = description
        self.__tags = tags
        self.__duration = duration
        self.__licensed_content = licensedContent
        
    def get_video_details(self):
        video_details = {
            'id': self.__id,
            'channel_id': self.__channel_id,
            'title': self.__title,
            'channel_title': self.__channel_title,
            'description': self.__description,
            'tags': self.__tags,
            'duration': self.__duration,
            'licensed_content': self.__licensed_content
        }
        return video_details
    
    def get_video_id(self):
        return self.__id

In [134]:
class YouTubeComment:
    def __init__(self, id: str, videoId: str, totalReplyCount: str, textDisplay: str, 
                authorDisplayName: str, authorProfileImageUrl: str, authorChannelId: str,
                likeCount: str, publishedAt: str, updatedAt: str):
        self.__id = id
        self.__video_id = videoId
        self.__total_reply_count = totalReplyCount
        self.__text_display = textDisplay
        self.__author_display_name = authorDisplayName
        self.__author_profile_image_url = authorProfileImageUrl
        self.__author_channel_id = authorChannelId
        self.__like_count = int(likeCount)
        self.__published_at = publishedAt
        self.__updated_at = updatedAt
        
    def get_comment(self):
        comment = dict(
            id=self.__id,
            videoId=self.__video_id,
            totalReplyCount=self.__total_reply_count,
            textDisplay=self.__text_display,
            authorDisplayName = self.__author_display_name,
            authorProfileImageUrl = self.__author_profile_image_url,
            authorChannelId = self.__author_channel_id,
            likeCount = self.__like_count,
            publishedAt = self.__published_at,
            updatedAt = self.__updated_at
        )
        return comment
    
    def get_comment_text(self):
        comment_text = self.__text_display
        return comment_text
    
    def __str__(self):
        return self.get_comment_text()
    
    def __repr__(self):
        return f"YouTubeComment(id='{self.__id}', videoId='{self.__video_id}', \
        totalReplyCount={self.__total_reply_count})"

In [135]:
class YouTubeCommentThread:
    def __init__(self, video_id: str):
        self.__video_id = video_id
        
    def get_video_comments(self, youtube_client):
        """Get the top level comments for a video."""
        youtube_comments = self.__find_comments(youtube_client)
        youtube_comments = [self.__create_comment(comment) for comment in youtube_comments]
        return youtube_comments
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            videoId=self.__video_id,
            part='snippet,replies'
        ) 
        return basic_info_params
    
    def __find_comments(self, youtube_client):
        """Find the video comments."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.commentThreads().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        comments = self.__parse_comments(search_response)
        return comments
    
    def __create_comment(self, comment_details):
        youtube_comment = YouTubeComment(**comment_details)
        return youtube_comment

    def __parse_comments(self, search_response):
        items = search_response['items']
        comments = []
        for item in items:
            comments.append({
                'id': item['id'],
                'videoId': item['snippet']['videoId'],
                'totalReplyCount': item['snippet']['totalReplyCount'],
                'textDisplay': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                'authorDisplayName': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'authorProfileImageUrl': item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl'],
                'authorChannelId': item['snippet']['topLevelComment']['snippet']['authorChannelId']['value'],
                'likeCount': item['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': item['snippet']['topLevelComment']['snippet']['updatedAt']
        })
        return comments

In [136]:
class YouTubeVideo:
    """A YouTube Video."""
    def __init__(self, video_details):
        self.__video_stats = self.__create_video_stats(video_details)
        self.__video_details = self.__create_video_details(video_details)
        self.__video_top_level_comments = None
        
    def get_video_stats_details(self):
        video_stats_details = dict()
        video_stats_details['details'] = self.get_video_details()
        video_stats_details['statistics'] = self.get_video_stats()
        return video_stats_details
    
    def get_video_comments(self, youtube_client):
        if not self.__video_top_level_comments:
            youtube_commenthread = YouTubeCommentThread(self.get_video_id())
            self.__video_top_level_comments = youtube_commenthread.get_video_comments(youtube_client)
        return self.__video_top_level_comments
        
    def __create_video_stats(self, video_details: dict):
        video_stats = YouTubeVideoStats(**video_details['statistics'])
        return video_stats
    
    def __create_video_details(self, video_details: dict):
        video_details = YouTubeVideoDetails(**video_details['details'])
        return video_details
        
    def get_video_stats(self):
        return self.__video_stats.get_video_stats()
    
    def get_video_details(self):
        return self.__video_details.get_video_details()
    
    def get_video_top_level_comments(self):
        pass
    
    def get_video_id(self):
        return self.__video_details.get_video_id()
    
    def get_video_title(self):
        pass

In [137]:
class FindVideo:
    def __init__(self, video_id: str):
        """Find the video with the given id."""
        self.__video_id = video_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        youtube_video = YouTubeVideo(parsed_response)
        return youtube_video
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        if items['snippet'].get('tags'):
            parsed_video_details['details']['tags'] = items['snippet']['tags']
        else:
            parsed_video_details['details']['tags'] = []
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [138]:
class VideoSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = ''
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
            regionCode=self.__REGION_CODE
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_related_video(self, video_id):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            relatedToVideoId=video_id,
            maxResults=self.__MAX_RESULTS,
            regionCode=self.__REGION_CODE
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_most_popular_video_by_region(self, 
                region_code):
        basic_info_params = dict(
            part='id,snippet',
            chart='mostPopular',
            regionCode=region_code
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_most_popular_video_by_category(self, 
                category_id):
        basic_info_params = dict(
            part='id,snippet',
            chart='mostPopular',
            videoCategoryId=category_id
        ) 
        return basic_info_params
    
    def search_video(self, query_string: str, youtube_client, search_type='basic'):
        self.__query = YouTubeSearchQuery(query_string)
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response(search_response, youtube_client)
        return videos
    
    def __parse_basic_response(self, search_response, youtube_client):
        videos = []
        video_results = search_response['items']
        for video_result in video_results:
            video_id = video_result['id']['videoId']
            youtube_video = FindVideo(video_id).find_video(youtube_client)
            videos.append(youtube_video)
        return videos
    
    def __parse_basic_response_popular_videos(self, search_response, youtube_client):
        videos = []
        video_results = search_response['items']
        for video_result in video_results:
            if video_result.get('id'):
                video_id = video_result['id']
            else:
                video_id = video_result['id']['videoId']
            youtube_video = FindVideo(video_id).find_video(youtube_client)
            videos.append(youtube_video)
        return videos
    
    def search_related_videos(self, youtube_video, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_related_video(youtube_video.get_video_id())
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response(search_response, youtube_client)
        return videos
    
    def search_most_popular_videos_by_region(self, region_code, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_most_popular_video_by_region(region_code)
            search_request = youtube_client.videos().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response_popular_videos(search_response, youtube_client)
        return videos
    
    def search_most_popular_videos_by_category(self, category_id, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_most_popular_video_by_category(category_id)
            search_request = youtube_client.videos().list(
                **basic_info_params
            )
            try:
                search_response = search_request.execute()
                videos = self.__parse_basic_response_popular_videos(search_response, youtube_client)
            except:
                videos = []
        return videos

In [139]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = self.__youtube_client = self.__auth.authenticate()

    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        videos = VideoSearch().search_video(query_string, self.__youtube_client)
        return videos
    
    def find_related_videos(self, youtube_video):
        related_videos = VideoSearch().search_related_videos(youtube_video, self.__youtube_client)
        return related_videos
    
    def find_most_popular_videos_by_region(self, region_code):
        most_popular_videos_by_region = VideoSearch().search_most_popular_videos_by_region(
            region_code, self.__youtube_client)
        return most_popular_videos_by_region
    
    def find_most_popular_videos_by_category(self, category_id):
        most_popular_videos_by_category = VideoSearch().search_most_popular_videos_by_category(
            category_id, self.__youtube_client)
        return most_popular_videos_by_category
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [16]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube_client = youtube.get_youtube()

In [17]:
videos = youtube.search_video('python programming')

In [18]:
videos

In [23]:
related_videos = youtube.search_related_videos(videos[0])

In [24]:
related_videos

In [25]:
related_videos[0].get_video_comments(youtube_client)

[YouTubeComment(id='UgzaJtLt9dkuVnCR1xR4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=8),
 YouTubeComment(id='UgySi_5On-yk6MEmBYt4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='Ugy5hawTDEWDcU9mJS94AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='Ugz2L363BYSXvft6mut4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='UgwX4ZbvM3aF1MtZAxl4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='Ugw97NdTMMK13TwMU6x4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='Ugw3tny8y5iuRg6oSSl4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='UgxuX5osaqiVuJCvjNF4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='UgyLgPEsPGJL_DPUVtt4AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),
 YouTubeComment(id='UgyIRGBEJDowZoX5zF54AaABAg', videoId='b093aqAZiPU',         totalReplyCount=0),


In [62]:
most_popular_ke_videos = youtube.find_most_popular_videos_by_region('ke')

In [63]:
most_popular_ke_videos

In [64]:
most_popular_ke_videos[-1].get_video_stats_details()

{'details': {'id': 'ksBpvcbjTXM',
  'channel_id': 'UCi_szzLDh7zYETJevZY0HqQ',
  'title': 'Kaskie Vibaya by Fathermoh & Ssaru',
  'channel_title': 'Fathermoh',
  'description': '"Kaskie Vibaya" by Fathermoh & Ssaru\nhttps://blackmarket.lnk.to/kaskievibaya\n#blackmarketrecords #fathermoh #kaskievibaya #ssaru',
  'tags': ['gang',
   'joefes',
   'wagithomo',
   'kashkeed',
   'sharma shamra',
   'jovial',
   'iphoolish',
   'fathermoh',
   'mejja',
   'sherehe Sheria',
   'adaa',
   'black market records',
   'gengetone',
   'gengetone fire',
   'bambi',
   'betty',
   'kenya music',
   'goats',
   'goat gang',
   'mbuzi gang music',
   'mbuzi gang spotify',
   'shamra shamra amapiano remix',
   'Amapiano remix',
   'dj ken skilla',
   'mr nun',
   'moyenga',
   'mr london',
   'kash keed',
   'studio black market kenya',
   'taki taki',
   'lamaz span kob',
   'mr. nun',
   'ngomma',
   'swahili music',
   'kiswahili muziki',
   'muziki',
   'father moh',
   'father-mo',
   'father-moh']

In [105]:
youtube.get_video_categories()

AttributeError: 'YouTube' object has no attribute 'get_video_categories'

In [140]:
class YouTubeVideoCategory:
    def __init__(self, id, title):
        self.__id = id
        self.__title = title
        
    def get_id(self):
        return self.__id
    
    def get_title(self):
        return self.__title
    
    def __str__(self):
        return self.get_title()
    
    def __repr__(self):
        return f"YouTubeVideoCategory(id='{self.__id}', title='{self.__title}')"

In [141]:
class SearchYouTubeVideoCategories:
    def __init__(self, youtube_client, region_code='us'):
        self.__youtube_client = youtube_client
        self.__region_code = region_code
        self.__youtube_video_categories = self.__search_video_categories()
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='snippet',
            regionCode=self.__region_code
        )
        return basic_info_params
    
    def __search_video_categories(self):
        basic_info_params = self.__generate_basic_info_params()
        search_request = self.__youtube_client.videoCategories().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        video_categories = self.__parse_categories(search_response)
        return video_categories
    
    def __parse_categories(self, search_response):
        category_data = []
        items = search_response['items']
        for item in items:
            category = dict()
            category['id'] = item['id']
            category['title'] = item['snippet']['title']
            category_data.append(YouTubeVideoCategory(**category))
        return category_data
    
    def get_youtube_video_categories(self):
        return self.__youtube_video_categories

In [94]:
video_categories = SearchYouTubeVideoCategories(youtube_client).get_youtube_video_categories()

In [95]:
video_categories

[YouTubeVideoCategory(id='1', title='Film & Animation'),
 YouTubeVideoCategory(id='2', title='Autos & Vehicles'),
 YouTubeVideoCategory(id='10', title='Music'),
 YouTubeVideoCategory(id='15', title='Pets & Animals'),
 YouTubeVideoCategory(id='17', title='Sports'),
 YouTubeVideoCategory(id='18', title='Short Movies'),
 YouTubeVideoCategory(id='19', title='Travel & Events'),
 YouTubeVideoCategory(id='20', title='Gaming'),
 YouTubeVideoCategory(id='21', title='Videoblogging'),
 YouTubeVideoCategory(id='22', title='People & Blogs'),
 YouTubeVideoCategory(id='23', title='Comedy'),
 YouTubeVideoCategory(id='24', title='Entertainment'),
 YouTubeVideoCategory(id='25', title='News & Politics'),
 YouTubeVideoCategory(id='26', title='Howto & Style'),
 YouTubeVideoCategory(id='27', title='Education'),
 YouTubeVideoCategory(id='28', title='Science & Technology'),
 YouTubeVideoCategory(id='29', title='Nonprofits & Activism'),
 YouTubeVideoCategory(id='30', title='Movies'),
 YouTubeVideoCategory(id='

In [142]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = self.__youtube_client = self.__auth.authenticate()
        self.__video_categories = []

    def get_youtube(self):
        return self.__youtube_client
    
    def get_video_categories(self):
        if not self.__video_categories:
            self.__video_categories = SearchYouTubeVideoCategories(self.__youtube_client).get_youtube_video_categories()
        return self.__video_categories
    
    def search_video(self, query_string: str) -> list[str]:
        videos = VideoSearch().search_video(query_string, self.__youtube_client)
        return videos
    
    def find_related_videos(self, youtube_video):
        related_videos = VideoSearch().search_related_videos(youtube_video, self.__youtube_client)
        return related_videos
    
    def find_most_popular_videos_by_region(self, region_code):
        most_popular_videos_by_region = VideoSearch().search_most_popular_videos_by_region(
            region_code, self.__youtube_client)
        return most_popular_videos_by_region
    
    def find_most_popular_videos_by_category(self, category_id):
        most_popular_videos_by_category = VideoSearch().search_most_popular_videos_by_category(
            category_id, self.__youtube_client)
        return most_popular_videos_by_category
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [121]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube_client = youtube.get_youtube()

In [122]:
youtube.get_video_categories()

[YouTubeVideoCategory(id='1', title='Film & Animation'),
 YouTubeVideoCategory(id='2', title='Autos & Vehicles'),
 YouTubeVideoCategory(id='10', title='Music'),
 YouTubeVideoCategory(id='15', title='Pets & Animals'),
 YouTubeVideoCategory(id='17', title='Sports'),
 YouTubeVideoCategory(id='18', title='Short Movies'),
 YouTubeVideoCategory(id='19', title='Travel & Events'),
 YouTubeVideoCategory(id='20', title='Gaming'),
 YouTubeVideoCategory(id='21', title='Videoblogging'),
 YouTubeVideoCategory(id='22', title='People & Blogs'),
 YouTubeVideoCategory(id='23', title='Comedy'),
 YouTubeVideoCategory(id='24', title='Entertainment'),
 YouTubeVideoCategory(id='25', title='News & Politics'),
 YouTubeVideoCategory(id='26', title='Howto & Style'),
 YouTubeVideoCategory(id='27', title='Education'),
 YouTubeVideoCategory(id='28', title='Science & Technology'),
 YouTubeVideoCategory(id='29', title='Nonprofits & Activism'),
 YouTubeVideoCategory(id='30', title='Movies'),
 YouTubeVideoCategory(id='

In [123]:
film_animation = youtube.find_most_popular_videos_by_category('1')

In [124]:
film_animation

In [125]:
film_animation[-1].get_video_stats_details()

{'details': {'id': '5f-KwlzsOe4',
  'channel_id': 'UCkj8f9sqSKuzn-NyP7Os1ZQ',
  'title': 'Buying Pretty Girls Whatever They Want😍',
  'channel_title': 'Jack Doherty',
  'description': 'Buying Pretty Girls Whatever They Want😍\n🔥 FOLLOW ME 🔥\nInstagram: https://www.instagram.com/jackdoherty\nTikTok: https://www.tiktok.com/@Jackdoherty',
  'tags': ['jack doherty tiktok',
   'tiktok',
   'pranks',
   'girlfriend',
   'funny',
   'family friendly',
   'prank',
   'relationships',
   'couples',
   'shorts',
   'jack doherty',
   'tik tok',
   'tiktok compilation',
   'tiktoks',
   'jackdoherty',
   'relatable'],
  'duration': 'PT42S',
  'licensed_content': True},
 'statistics': {'view_count': 2265395,
  'like_count': 141808,
  'comment_count': 606}}

In [126]:
movies = youtube.find_most_popular_videos_by_category('39')

In [127]:
movies

[]

In [115]:
movies[-1].get_video_stats_details()

{'details': {'id': '5sLzyEzUbPo',
  'channel_id': 'UCRI7hheejBbWS6etTNwMT0g',
  'title': 'Anuel AA, Mambo Kingz, DJ Luian - Mas Rica Que Ayer (Video Oficial)',
  'channel_title': 'Anuel AA',
  'description': 'Anuel AA, Mambo Kingz, DJ Luian - Mas Rica Que Ayer (Video Oficial)\n\nSuscríbete a mi canal ➡️   https://radi.al/AnuelAA \n\n🎵 Stream/Download: https://orcd.co/masricaqueayer\n\nRedes Sociales\nhttps://www.facebook.com/Anuel2bleAofficial/\nhttps://twitter.com/_anuel_aa\nhttps://www.instagram.com/anuel/\n\nhttps://www.anuelaa.com\n\n#anuelaa  #MasRicaQueAyer  #nuevamusica',
  'tags': ['Anuel AA',
   'anuel',
   'Free anuel',
   'Emmanuel',
   'Real hasta la muerte',
   'New Music',
   'Reggaeton',
   'Trap',
   'Latin Trap',
   '2022',
   'Las Leyendas Nunca Mueren',
   'Las Leyendas Nunca Mueren 2',
   'LLNM',
   'LLNM2',
   'anuel aa',
   'ANUEL',
   'new album',
   'nuevo album',
   'Mambo kingz',
   'dj luian',
   'maluma'],
  'duration': 'PT3M47S',
  'licensed_content': True}

In [18]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube_client = youtube.get_youtube()

In [143]:
class PlaylistItem:
    def __init__(self, id, publishedAt, channelId, title, description, thumbnails, 
                channelTitle, position, videoId, videoOwnerChannelTitle, 
                videoOwnerChannelId):
        self.__id = id
        self.__publishedAt = publishedAt
        self.__channelId = channelId
        self.__title = title
        self.__description = description
        self.__thumbnails = thumbnails
        self.__channelTitle = channelTitle
        self.__position = int(position)
        self.__videoId = videoId
        self.__videoOwnerChannelTitle = videoOwnerChannelTitle
        self.__videoOwnerChannelId = videoOwnerChannelId
        self.__video = None
        
    def get_video(self, youtube_client):
        if not self.__video:
            self.__video = FindVideo(self.__videoId).find_video(youtube_client)
        return self.__video

In [171]:
class PlayList:
    def __init__(self, id, channelId, title, description, thumbnails, channelTitle, 
                itemCount, player):
        self.__id = id
        self.__channelId = channelId
        self.__title = title
        self.__description = description
        self.__thumbnails = thumbnails
        self.__channelTitle = channelTitle
        self.__itemCount = itemCount
        self.__player = player
        self.__playlist_items = []
        self.__videos = []
        
    def get_playlist_items(self, youtube_client):
        if not self.__playlist_items:
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.playlistItems().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            parsed_response = self.__parse_playlist_items(search_response)
            self.__playlist_items = [PlaylistItem(**item) for item in parsed_response]
        return self.__playlist_items
    
    def get_videos(self, youtube_client):
        if not self.__videos:
            play_list_items = self.get_playlist_items(youtube_client)
            for playlist_item in play_list_items:
                self.__videos.append(playlist_item.get_video(youtube_client))
        return self.__videos
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet,contentDetails',
            playlistId=self.__id
        ) 
        return basic_info_params
    
    def __parse_playlist_items(self, search_response):
        playlist_items = []
        items = search_response['items']
        for item in items:
            playlist_item = dict()
            playlist_item['id'] = item['id']
            playlist_item['publishedAt'] = item['snippet']['publishedAt']
            playlist_item['channelId'] = item['snippet']['channelId']
            playlist_item['title'] = item['snippet']['title']
            playlist_item['description'] = item['snippet']['description']
            playlist_item['thumbnails'] = item['snippet']['thumbnails']
            playlist_item['channelTitle'] = item['snippet']['channelTitle']
            playlist_item['position'] = item['snippet']['position']
            playlist_item['videoId'] = item['snippet']['resourceId']['videoId']
            playlist_item['videoOwnerChannelTitle'] = item['snippet']['videoOwnerChannelTitle']
            playlist_item['videoOwnerChannelId'] = item['snippet']['videoOwnerChannelId']
            playlist_items.append(playlist_item)
        return playlist_items
    
    def get_playlist_thumbnail(self):
        return self.__thumbnails['standard']['url']
    
    def get_playlist_title(self):
        return self.__title
    
    def get_playlist_channel_title(self):
        pass

In [174]:
class FindPlaylist:
    def __init__(self, playlist_id: str):
        """Find the video with the given id."""
        self.__playlist_id = playlist_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__playlist_id,
            part='id,snippet,contentDetails,player',
        ) 
        return basic_info_params
    
    def find_playlist(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.playlists().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_playlist(search_response)
        youtube_playlist = PlayList(**parsed_response)
        return youtube_playlist
    
    def __parse_playlist(self, search_response):
        playlist_details = dict()
        items = search_response['items'][0]
        playlist_details['id'] = items['id']
        playlist_details['channelId'] = items['snippet']['channelId']
        playlist_details['title'] = items['snippet']['title']
        playlist_details['description'] = items['snippet']['description']
        playlist_details['thumbnails'] = items['snippet']['thumbnails']
        playlist_details['channelTitle'] = items['snippet']['channelTitle']
        playlist_details['itemCount'] = items['contentDetails']['itemCount']
        playlist_details['player'] = items['player']['embedHtml']
        return playlist_details

In [175]:
playlist = FindPlaylist('PLsyeobzWxl7poL9JTVyndKe62ieoN-MZ3').find_playlist(youtube_client)

In [176]:
playlist.get_playlist_thumbnail()

'https://i.ytimg.com/vi/QXeEoD0pB3E/sddefault.jpg'

In [167]:
playlist

{'id': 'PLsyeobzWxl7poL9JTVyndKe62ieoN-MZ3',
 'channelId': 'UC59K-uG2A5ogwIrHw4bmlEg',
 'title': 'Python for Beginners (Full Course) | Programming Tutorial',
 'description': 'Python Tutorial, Easy Python tutorial for beginner, learn Python Programming, learn python programming with example and syntax. Download python, install python, using python IDE, IDLE, Pycharm IDE, notepad,  getting started with python, variables, functions, object, list in python, set in python, help in python, data types, operators, math function, run, debug python code, conditional statement, array, matrix in python, python by navin reddy',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/default.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'high': {'url': 'https://i.ytimg.com/vi/QXeEoD0pB3E/hqdefault.jpg',
   'width': 480,
   'height': 360},
  'standard': {'url': 'https://i.ytimg.com/

In [170]:
playlist['thumbnails']['standard']['url']

'https://i.ytimg.com/vi/QXeEoD0pB3E/sddefault.jpg'

In [146]:
class PlaylistSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.PLAYLIST
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
        ) 
        return basic_info_params
    
    def search_playlist(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            playlist_ids = self.__parse_playlists(search_response)
            play_lists = [FindPlaylist(playlist_id).find_playlist(youtube_client)
                         for playlist_id in playlist_ids]
        return play_lists
    
    def __parse_playlists(self, search_response):
        playlists_ids = []
        items = search_response['items']
        for item in items:
            playlists_ids.append(item['id']['playlistId'])
        return playlists_ids
        

In [44]:
playlists = PlaylistSearch('python programming').search_playlist(youtube_client)

In [147]:
class Channel:
    def __init__(self, id, title, description, customUrl, publishedAt, thumbnails, statistics):
        self.__id = id
        self.__title = title
        self.__description = description
        self.__customUrl = customUrl
        self.__publishedAt = publishedAt
        self.__thumbnails = thumbnails
        self.__statistics = statistics

In [153]:
class FindChannel:       
    def __generate_basic_info_params(self, channel_id):
        basic_info_params = dict(
            id=channel_id,
            part='id,snippet,contentDetails,contentOwnerDetails,statistics,topicDetails',
        ) 
        return basic_info_params
    
    def find_channel_by_name(self, youtube_client):
        pass
    
    def find_channel_by_id(self, channel_id, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params(channel_id)
        search_request = youtube_client.channels().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_channel(search_response)
        youtube_channel = Channel(**parsed_response)
        return youtube_channel
    
    def __parse_channel(self, search_response):
        channel_details = {}
        items = search_response['items'][0]
        channel_details['id'] = items['id']
        channel_details['title'] = items['snippet']['title']
        channel_details['description'] = items['snippet']['description']
        channel_details['customUrl'] = items['snippet']['customUrl']
        channel_details['publishedAt'] = items['snippet']['publishedAt']
        channel_details['thumbnails'] = items['snippet']['thumbnails']
        channel_details['statistics'] = dict()
        channel_details['statistics']['viewCount'] = items['statistics']['viewCount']
        channel_details['statistics']['subscriberCount'] = items['statistics']['subscriberCount']
        channel_details['statistics']['videoCount'] = items['statistics']['videoCount']
        return channel_details

In [154]:
channel = FindChannel().find_channel_by_id('UC_q1lWD0uKH8tMsX8w_u94g', youtube_client)

In [155]:
channel

In [160]:
class ChannelSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.CHANNEL
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
        ) 
        return basic_info_params
    
    def search_channels(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            channel_ids = self.__parse_channels(search_response)
            channels = [FindChannel().find_channel_by_id(channel_id, youtube_client)
                         for channel_id in channel_ids]
        return channels
    
    def __parse_channels(self, search_response):
        channels_ids = []
        items = search_response['items']
        for item in items:
            channels_ids.append(item['id']['channelId'])
        return channels_ids
        

In [161]:
channels = ChannelSearch('python programming').search_channels(youtube_client)

In [162]:
channels